# Anti-Aliasing Neural Network Training on Google Colab

This notebook enables GPU-accelerated training of anti-aliasing RNN models.

**Expected speedup:** 20-30x (from 32 min/epoch on CPU to ~1-2 min/epoch on T4 GPU)

## Setup Overview

1. Mount Google Drive for persistent storage
2. Clone repository from GitHub
3. Install dependencies
4. Create symlinks for weights/audio/checkpoints
5. Configure wandb for remote monitoring
6. Run training

## Before Running

Ensure your Google Drive has this structure:
```
Google Drive/
└── AA_Neural/
    ├── weights/
    │   └── NAM/
    │       └── Marshall JCM 800 2203/
    │           └── JCM800 2203 - P5 B5 M5 T5 MV7 G10 - AZG - 700.nam
    ├── audio_data/
    │   └── val_input.wav
    └── checkpoints/
        (created automatically)
```

## 1. Mount Google Drive

This mounts your Google Drive to `/content/drive` for persistent storage.

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Verify the AA_Neural folder exists
drive_base = '/content/drive/MyDrive/AA_Neural'
assert os.path.exists(drive_base), f"ERROR: {drive_base} not found. Create it and upload weights/audio_data first."
assert os.path.exists(f'{drive_base}/weights'), f"ERROR: {drive_base}/weights not found."
assert os.path.exists(f'{drive_base}/audio_data'), f"ERROR: {drive_base}/audio_data not found."

print("✓ Google Drive mounted successfully")
print(f"✓ Found {drive_base}")
print(f"✓ Found {drive_base}/weights")
print(f"✓ Found {drive_base}/audio_data")